# Objective
- Check graphSage works by 

In [10]:
import torch

print(torch.__version__)

2.0.0+cpu


In [11]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Feb__8_05:53:42_Coordinated_Universal_Time_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0


In [9]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.0.0+cu121.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.0.0+cu121.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu121.html
  Using cached torch_scatter-2.1.1.tar.gz (107 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for torch-scatter: filename=torch_scatter-2.1.1-cp311-cp311-win_amd64.whl size=500932 sha256=57883e7469ec64d979d73f28374bf49eb5909f0e3f17d02ac1cbab83fd4457fb
  Stored in directory: c:\users\mnihy\appdata\local\pip\cache\wheels\e1\45\de\7e6c2b34bf0c92ea931392eb9930fa25ac12bf455e68ae1d6e
Successfully built torch-scatter
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu121.html
  Using cached torch_sparse-0.6.17.tar.gz (209 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for torch-sparse: filename=torch_sparse-0.6.17-cp311-cp311-win_amd64.whl size=1178379 sha256=32bb0ccde3ebbf8ce0901256d1bc79359fde01a875ea17d8cbee17e557d01b3f
  Stored in directory: c:\users\mn

In [12]:
xx# read the pickled data from generateData folder
import pickle

adjacencyList = pickle.load(open("../generateData/adjacencyList.pkl", "rb"))x

In [13]:
adjacencyList

tensor([[ 0,  1],
        [ 0,  2],
        [ 0,  3],
        [ 0,  4],
        [ 1,  5],
        [ 1,  6],
        [ 1,  7],
        [ 1,  8],
        [ 2,  9],
        [ 2, 10],
        [ 2, 11],
        [ 2, 12],
        [ 3, 13],
        [ 3, 14],
        [ 3, 15],
        [ 3, 16],
        [ 4, 17],
        [ 4, 18],
        [ 4, 19],
        [ 4, 20]], dtype=torch.int32)

In [10]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

# Load the Cora dataset and apply normalization transform
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=T.NormalizeFeatures())
data = dataset[0]

class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphSAGE(dataset.num_features, 16, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
    return correct / data.test_mask.sum().item()

for epoch in range(1, 201):
    loss = train()
    if epoch % 10 == 0:
        acc = test()
        print(f'Epoch: {epoch}, Loss: {loss:.4f}, Test Acc: {acc:.4f}')

Epoch: 10, Loss: 1.8202, Test Acc: 0.4770


KeyboardInterrupt: 

In [26]:
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()

In [25]:
# read the pickled data from generateData folder
import pickle

adjacencyList = pickle.load(open("../generateData/adjacencyList.pkl", "rb"))
nodeLabels = pickle.load(open("../generateData/nodeLabels.pkl", "rb"))

In [15]:
# data.x is feature matrix. Use nodeLabes instead
# data.edge_index is edge list. Use adjacencyList instead

# 


7

In [23]:
out = model(data.x, data.edge_index)
pred = out.argmax(dim=1)
# check the unique values in pred
pred.unique()

tensor([0, 1, 2, 3, 4, 5, 6])